In [2]:
import sys
import os

# Agregar la ruta del directorio donde están los módulos
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [4]:
from model import FastSpeech2
import yaml

In [14]:

base_path = "C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\config\\F-VOICE\\"

preprocess_config = yaml.load(open(base_path+"preprocess.yaml", "r"), Loader=yaml.FullLoader)
model_config = yaml.load(open(base_path+"model.yaml", "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(base_path+"train.yaml", "r"), Loader=yaml.FullLoader)

In [15]:
configs = (preprocess_config, model_config, train_config)
model = FastSpeech2(preprocess_config, model_config)



### **Capas para Congelar (Freeze)**
Estas capas son menos dependientes del hablante y suelen encargarse de transformar el texto en representaciones internas o realizar tareas de bajo nivel que no necesitan ser reentrenadas para una nueva voz.

1. **`encoder`**
   - **`src_word_emb`**: Embedding de palabras o fonemas. Esta capa no depende del hablante, por lo que puedes congelarla.
   - **`layer_stack` (todas las capas FFTBlock dentro del encoder)**: Estas capas procesan las representaciones de texto. Puedes congelarlas ya que no dependen del timbre o las características específicas del hablante.

2. **`variance_adaptor`**
   - **`pitch_embedding`** y **`energy_embedding`**: Estas embeddings se utilizan para la modulación del tono y la energía, y no necesitan ser ajustadas en la mayoría de los casos, por lo que puedes congelarlas.

### **Capas para Descongelar (Fine-tuning)**
Estas capas afectan directamente las características acústicas y prosódicas de la síntesis de voz, por lo que debes ajustarlas al nuevo hablante.

1. **`variance_adaptor`**
   - **`duration_predictor`**, **`pitch_predictor`**, **`energy_predictor`**: Estas capas controlan la duración, el tono y la energía de la voz, que son características críticas para la adaptación de la nueva voz, por lo que deben ser ajustadas. Descongélalas para que el modelo pueda aprender la prosodia específica del hablante.
   
2. **`decoder`**
   - **`layer_stack` (todas las capas FFTBlock dentro del decoder)**: Estas capas convierten las representaciones internas en espectrogramas de salida, los cuales dependen directamente del estilo y las características del hablante. Es importante descongelarlas para que el modelo pueda ajustar estas capas a la nueva voz.
   
3. **`mel_linear`**: La capa que convierte la salida del decoder en espectrogramas mel. Descongelarla permitirá ajustar mejor la salida del modelo a las características del hablante.

4. **`postnet`**: Esta red post-procesa el espectrograma generado por el modelo para mejorar la calidad de la salida de audio. Debes descongelar esta capa para ajustar el modelo a la nueva voz y obtener mejores resultados en la síntesis de audio.

### **Resumen de Capas a Congelar y Descongelar:**

- **Congelar**:
  - **`encoder`** (todas las capas, incluyendo embeddings y FFTBlock)
  - **`variance_adaptor.pitch_embedding`** y **`variance_adaptor.energy_embedding`**

- **Descongelar**:
  - **`variance_adaptor.duration_predictor`**, **`pitch_predictor`**, **`energy_predictor`**
  - **`decoder.layer_stack` (todas las FFTBlock)**
  - **`mel_linear`**
  - **`postnet`**

Este enfoque permite que el modelo mantenga sus representaciones generales de texto y fonemas, mientras ajusta las capas necesarias para generar las características de la voz y la prosodia del nuevo hablante.


In [17]:
print(model)

FastSpeech2(
  (encoder): Encoder(
    (src_word_emb): Embedding(120, 256, padding_idx=0)
    (layer_stack): ModuleList(
      (0-3): 4 x FFTBlock(
        (slf_attn): MultiHeadAttention(
          (w_qs): Linear(in_features=256, out_features=256, bias=True)
          (w_ks): Linear(in_features=256, out_features=256, bias=True)
          (w_vs): Linear(in_features=256, out_features=256, bias=True)
          (attention): ScaledDotProductAttention(
            (softmax): Softmax(dim=2)
          )
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (fc): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (pos_ffn): PositionwiseFeedForward(
          (w_1): Conv1d(256, 1024, kernel_size=(9,), stride=(1,), padding=(4,))
          (w_2): Conv1d(1024, 256, kernel_size=(1,), stride=(1,))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout): Dr

In [24]:
model.encoder

Encoder(
  (src_word_emb): Embedding(120, 256, padding_idx=0)
  (layer_stack): ModuleList(
    (0-3): 4 x FFTBlock(
      (slf_attn): MultiHeadAttention(
        (w_qs): Linear(in_features=256, out_features=256, bias=True)
        (w_ks): Linear(in_features=256, out_features=256, bias=True)
        (w_vs): Linear(in_features=256, out_features=256, bias=True)
        (attention): ScaledDotProductAttention(
          (softmax): Softmax(dim=2)
        )
        (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (pos_ffn): PositionwiseFeedForward(
        (w_1): Conv1d(256, 1024, kernel_size=(9,), stride=(1,), padding=(4,))
        (w_2): Conv1d(1024, 256, kernel_size=(1,), stride=(1,))
        (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
  )
)

In [30]:
# Congelar todas las capas del encoder
for param in model.encoder.parameters():
    param.requires_grad = False

In [31]:
model.variance_adaptor

VarianceAdaptor(
  (duration_predictor): VariancePredictor(
    (conv_layer): Sequential(
      (conv1d_1): Conv(
        (conv): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (relu_1): ReLU()
      (layer_norm_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout_1): Dropout(p=0.5, inplace=False)
      (conv1d_2): Conv(
        (conv): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (relu_2): ReLU()
      (layer_norm_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout_2): Dropout(p=0.5, inplace=False)
    )
    (linear_layer): Linear(in_features=256, out_features=1, bias=True)
  )
  (length_regulator): LengthRegulator()
  (pitch_predictor): VariancePredictor(
    (conv_layer): Sequential(
      (conv1d_1): Conv(
        (conv): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (relu_1): ReLU()
      (layer_norm_1): LayerNorm((256,), eps=1e-05, elementwise_affine=

In [32]:
# Congelar las embeddings de pitch y energía
model.variance_adaptor.pitch_embedding.weight.requires_grad = False
model.variance_adaptor.energy_embedding.weight.requires_grad = False

In [33]:
model.decoder

Decoder(
  (layer_stack): ModuleList(
    (0-5): 6 x FFTBlock(
      (slf_attn): MultiHeadAttention(
        (w_qs): Linear(in_features=256, out_features=256, bias=True)
        (w_ks): Linear(in_features=256, out_features=256, bias=True)
        (w_vs): Linear(in_features=256, out_features=256, bias=True)
        (attention): ScaledDotProductAttention(
          (softmax): Softmax(dim=2)
        )
        (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (pos_ffn): PositionwiseFeedForward(
        (w_1): Conv1d(256, 1024, kernel_size=(9,), stride=(1,), padding=(4,))
        (w_2): Conv1d(1024, 256, kernel_size=(1,), stride=(1,))
        (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
  )
)

In [ ]:
# Optimizar solo los parámetros que se van a entrenar
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)